# Traitement des données

In [ ]:
import numpy as np
import pylab as pl
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

X_train = pd.read_csv('X_train_HI5.csv')
X_test = pd.read_csv('X_test_HI5.csv')

y_train = X_train['piezo_groundwater_level_category']
X_train = X_train.drop(['piezo_groundwater_level_category'], axis=1) 

## Cohérence des types

### Piezo variables

In [ ]:
# Filtrer les colonnes commençant par "piezo"
piezo_columns = [col for col in X_train.columns if col.startswith('piezo')]

# Vérifier les types des colonnes sélectionnées
piezo_types = X_train[piezo_columns].dtypes

# Afficher les résultats
piezo_types


**Dates (actuellement object) :**
Les colonnes piezo_station_update_date et piezo_measurement_date sont des dates, mais elles sont au format object.
Action : Convertir ces colonnes en type datetime.

In [ ]:
X_train['piezo_station_update_date'] = X_train['piezo_station_update_date'].str.replace(r'CEST', '', regex=True).str.strip()
X_train['piezo_measurement_date'] = X_train['piezo_measurement_date'].str.replace(r'CEST', '', regex=True).str.strip()
X_train['piezo_station_update_date'] = pd.to_datetime(X_train['piezo_station_update_date'], errors='coerce')
X_train['piezo_measurement_date'] = pd.to_datetime(X_train['piezo_measurement_date'], errors='coerce')


**Codes et identifiants (actuellement object) :** 

Colonnes concernées :  

piezo_station_department_code, piezo_station_bdlisa_codes, piezo_station_bss_code, piezo_station_bss_id, piezo_bss_code.  

Ces colonnes contiennent des codes ou identifiants. Si elles ne sont pas utilisées comme valeurs numériques, elles peuvent rester au format object. Sinon, vérifier s'il est pertinent de les convertir en category pour économiser de la mémoire.  

Action : Convertir en category si elles ne nécessitent pas d'opérations numériques.

In [ ]:
cols_to_category = [
    'piezo_station_department_code', 'piezo_station_bdlisa_codes', 
    'piezo_station_bss_code', 'piezo_station_bss_id', 'piezo_bss_code'
]
X_train[cols_to_category] = X_train[cols_to_category].astype('category')


**Catégories qualitatives (actuellement object) :**  

Colonnes concernées :  

piezo_station_department_name, piezo_station_commune_name, piezo_obtention_mode, piezo_status, piezo_qualification, piezo_continuity_name, piezo_producer_name, piezo_measure_nature_code, piezo_measure_nature_name.  

Ces colonnes semblent représenter des catégories qualitatives (ex. : nom de département, nature de la mesure).  

Action : Convertir en type category si elles sont destinées à être utilisées pour le regroupement, l'analyse ou la modélisation.

In [ ]:
qualitative_cols = [
    'piezo_station_department_name', 'piezo_station_commune_name', 
    'piezo_obtention_mode', 'piezo_status', 'piezo_qualification', 
    'piezo_continuity_name', 'piezo_producer_name', 
    'piezo_measure_nature_code', 'piezo_measure_nature_name'
]
X_train[qualitative_cols] = X_train[qualitative_cols].astype('category')


**Colonnes numériques (déjà float64)**

Colonnes concernées :   

piezo_station_investigation_depth, piezo_station_altitude, piezo_station_longitude, piezo_station_latitude, piezo_continuity_code, piezo_producer_code.   

Ces colonnes semblent correctement typées. Vérifie cependant la présence d'erreurs ou de valeurs aberrantes (NaN, valeurs extrêmes).   

Action : Analyser les valeurs aberrantes et les NaN pour décider d'un traitement adapté.

In [ ]:
print(X_train[['piezo_station_investigation_depth', 'piezo_station_altitude']].describe())


### Meteo variables

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.4f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [ ]:
# Filtrer les colonnes commençant par "meteo_"
meteo_columns = [col for col in X_train.columns if col.startswith('meteo_')]

# Vérifier les types des colonnes sélectionnées
meteo_types = X_train[meteo_columns].dtypes

# Calculer les statistiques descriptives pour les colonnes quantitatives
print_full(meteo_types)



**Dates (meteo_date)**

Actuellement au format object, ce qui limite les manipulations.  

Action : Convertir en type datetime pour permettre les analyses temporelles.

In [ ]:
X_train['meteo_date'] = pd.to_datetime(X_train['meteo_date'], errors='coerce')


**Identifiants et noms (meteo_id, meteo_name)**

meteo_id (station ID) : Correctement typé en int64.  

meteo_name (nom de la station) : Peut être converti en type category si utilisé pour le regroupement ou l'analyse.  

Action : Convertir meteo_name en category.

In [ ]:
X_train['meteo_name'] = X_train['meteo_name'].astype('category')


**3. Variables booléennes encodées en float64**

Colonnes : meteo_if_snow, meteo_if_fog, meteo_if_thunderstorm, etc. 

Ces colonnes semblent représenter des indicateurs booléens (présence/absence). 

Action : Convertir en type bool.

In [ ]:
bool_columns = [
    'meteo_if_snow', 'meteo_if_fog', 'meteo_if_thunderstorm', 'meteo_if_sleet', 'meteo_if_hail',
    'meteo_if_dew', 'meteo_if_black_ice', 'meteo_if_snow_ground', 'meteo_if_frost', 
    'meteo_if_smoke', 'meteo_if_mist', 'meteo_if_lightning'
]
X_train[bool_columns] = X_train[bool_columns].astype('bool')


**Variables quantitatives (float64)**

Les variables comme meteo_temperature_min, meteo_radiation, etc., sont correctement typées. 

Action : Vérifier les valeurs aberrantes et remplacer les NaN par des stratégies spécifiques (moyenne, interpolation, etc.).

In [ ]:
# Identifier les colonnes quantitatives
quantitative_vars = X_train[meteo_columns].select_dtypes(include=['float64', 'int64']).columns
print(quantitative_vars)

# Calculer les statistiques descriptives pour les colonnes quantitatives
print_full(X_train[quantitative_vars].describe())
